In [ ]:
!conda create -n macsima python=3.10
!conda activate macsima
!pip install -r requiremnts.txt

In [ ]:
# read in json file (macsima output)
# get important info into format that makes sense
# save to excel and/or report


- overall exp metadata
-name
- run time - start time - end time

- blocks
- cycles
- magnification
- colour names
- detection channel

- background pictures without any staining
- another can just the region of interest
- time coefficient for each cycle

In [1]:
import json
import pandas as pd

In [2]:
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

In [3]:
macsima_output_file = 'data/250128_macsima_output.json'
data = load_json(macsima_output_file)

In [50]:
data.keys()

dict_keys(['experiments', 'procedures', 'racks', 'reagents', 'rois', 'samples'])

In [13]:
data['experiments'][0]

{'actualRunningTime': 53367,
 'averageROIPerWell': 1,
 'averageSizePerROI': 1,
 'comment': '',
 'conclusion': '',
 'creationDateTime': '2025-01-28T15:50:01Z',
 'creator': '{23a8d883-dba3-42b1-8ec3-343c5f4137e0}',
 'datasetSize': -1,
 'estimatedRunningTime': 50983,
 'executionEndDateTime': '2025-01-29T06:43:08Z',
 'executionStartDateTime': '2025-01-28T15:53:36Z',
 'experimenter': 'admin',
 'id': '{9a237cfc-3501-4c97-a7d3-501016171e36}',
 'instrumentId': '{d2ccd42e-8e22-4da1-b82b-37a58b004efa}',
 'isArchived': False,
 'isBackedUp': False,
 'isBioQC': False,
 'isExpress': False,
 'keyword': '',
 'modificationDateTime': '2025-01-29T06:43:10Z',
 'modifier': '{23a8d883-dba3-42b1-8ec3-343c5f4137e0}',
 'name': '250128_liver_OCT_FCRB 1',
 'organization': '',
 'owner': '',
 'parameters': '',
 'plannedExecutionDateTime': '',
 'project': '',
 'purpose': '',
 'runExperimentPaths': ['/raid/shared/experiments/250128_liver_OCT_FCRB1_2025-01-28_15-53-43'],
 'sampleRackIds': ['{6959567b-27db-4395-8dd4-3

In [4]:
data.keys()

dict_keys(['experiments', 'procedures', 'racks', 'reagents', 'rois', 'samples'])

In [54]:
for rack in data['racks']:
    print(rack['name'])

250128_FCRB


In [15]:
len(data['rois'])

2

In [23]:
data['rois'][0]['shape']['Data']

'{"Height":10,"Width":19}'

In [24]:
import json

def extract_roi_dimensions(rois):
    dimensions = []
    for roi in rois:
        try:
            shape_data_str = roi.get('shape', {}).get('Data', '{}')
            shape_data = json.loads(shape_data_str)
            height = shape_data.get('Height')
            width = shape_data.get('Width')
            dimensions.append({'Height': height, 'Width': width})
        except (json.JSONDecodeError, TypeError) as e:
            dimensions.append({'Height': None, 'Width': None})
    return dimensions


In [25]:
roi_dimensions = extract_roi_dimensions(data['rois'])

for idx, dim in enumerate(roi_dimensions):
    print(f"ROI {idx+1}: Height = {dim['Height']}, Width = {dim['Width']}")


ROI 1: Height = 10, Width = 19
ROI 2: Height = 2.350855833425806, Width = 2.6345827695784165


In [11]:
data['racks'][0]['name']

'250128_FCRB'

In [12]:
len(data['racks'])

1

In [5]:
data['racks'][0].keys()
# slides
# 8 rois spread accros 4 racks
# which ROIs are from which rack

dict_keys(['analysisAlgorithmId', 'comment', 'contentType', 'creationDateTime', 'creator', 'id', 'isArchived', 'isTemplate', 'lotNumber', 'modificationDateTime', 'modifier', 'name', 'orderNumber', 'owner', 'preparationDateTime', 'qualityControlAlgorithmId', 'rackInfo', 'sampleGroupIds', 'sampleIds', 'scanDateTime', 'serialNumber', 'state', 'wells'])

In [6]:
data['experiments'][0]['name']

'250128_liver_OCT_FCRB 1'

In [ ]:
the sample info is associted with the rack_name
    and the 

within the ROI info we have a sample id

for each roi we want to know whuch spample its associated with

In [ ]:
data['rois'][1]

In [7]:
experiment_name = data['experiments'][0]['name']
# 250128_liver_OCT_FCRB 1

In [9]:
procedure_name = data['procedures'][0]['comment']
procedure_name
# standard procedure

'Standard procedure'

In [ ]:
data

In [ ]:
# I have only run with one "rack" so far, 
# but the expt we have running now has 2, 
# so I will see if the file has diff info
#  or diff layout that would be important
rack_name = data['racks'][0]['name']
# '250128_FCRB'

In [ ]:
#  'executionEndDateTime': '2025-01-29T06:43:08Z',
# 'executionStartDateTime': '2025-01-28T15:53:36Z',
start_time = data['experiments'][0]['executionStartDateTime']
end_time = data['experiments'][0]['executionEndDateTime']

# 'actualRunningTime':  53367,
actual_running_time_total_seconds = data['experiments'][0]['actualRunningTime']



In [ ]:
# Convert seconds to hours minutes and seconds from "actualRunningTime" which reports running time in seconds
def convert_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds_left = seconds % 60
    return hours, minutes, seconds_left

# 14 hrs 49 min 27 sec

In [ ]:
runtime_hours, runtime_minutes, runtime_seconds = convert_seconds_to_hms(actual_running_time_total_seconds)

In [ ]:
runtime_hours, runtime_minutes, runtime_seconds

In [ ]:
# usedDiskspace (I don't know what unit this is,
#  but I can check and then it would be good to report in GB)
# 'usedDiskspace': 178364,
used_disk_space = data['experiments'][0]['usedDiskspace']

In [ ]:
# I need to check the units; 
# also ROIS listed after reagents in json; 
# also note we usually have one ROI but we could have more
rois_shape_data = data['rois'][0]['shape']
rois_shape_type = data['rois'][0]['type']


In [26]:
data['samples'][0]['name']

'250128_liver_OCT_FCRB'

In [27]:
data['samples'][0]['species']

'Human'

In [28]:
data['samples'][0]['sampleType']

'SampleType_Tissue'

In [29]:
data['samples'][0]['organ']

'Liver'

In [30]:
data['samples'][0]['fixationMethod']

'PFA'

In [ ]:
sample_name = data['samples'][0]['name']
sample_species = data['samples'][0]['species']
sample_type = data['samples'][0]['sampleType']
sample_organ = data['samples'][0]['organ']
fixation_method = data['samples'][0]['fixationMethod']

In [ ]:
len(data['rois'])

In [ ]:
autofocus_methods = [data['rois'][i]['autoFocus']['method'] for i in range(len(data['rois']))]

In [51]:
data['procedures']

[{'blocks': [{'blockType': 'ProtocolBlockType_Scan',
    'comment': 'Scan',
    'creationDateTime': '',
    'creator': '',
    'id': '{7134d8cd-f0ef-40de-9d0a-4b2e8a6e1107}',
    'isArchived': False,
    'isEnabled': True,
    'isEntireWell': True,
    'isLocked': False,
    'magnification': 'Magnification_2x',
    'modificationDateTime': '',
    'modifier': '',
    'name': 'Scan',
    'owner': '',
    'reagents': {'DetectionChannel_1': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0,
        'sensitivityMode': 0},
       'exposureType': 'Coefficient',
       'timeCoefficient': 100},
      'fluorochromeType': 'FluorochromeType_DAPI',
      'isEnabled': True},
     'DetectionChannel_2': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0,
        'sensitivityMode': 0},
       'exposureType': 'Coefficient',
       'timeCoefficient': 230},
      'fluorochromeType': 'FluorochromeType_FITC',
      'isEnabled': False},
     'DetectionChannel_3': {'exposure

In [32]:
blocks = data['procedures'][0]['blocks']

In [33]:
len(blocks)

18

In [35]:
for block in blocks:
    print(block['blockType'])

ProtocolBlockType_Scan
ProtocolBlockType_DefineROIs
ProtocolBlockType_Scan
ProtocolBlockType_Erase
ProtocolBlockType_RestainNuclei
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle
ProtocolBlockType_RunCycle


In [39]:
for block in blocks:
    print(f"name: {block['name']}")
    print(f"comment: {block['comment']}")

name: Scan
comment: Scan
name: Define ROIs
comment: Define ROIs
name: Scan
comment: Scan
name: Erase
comment: Erase
name: Restain Nuclei
comment: Restain nuclei
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 
name: Run Cycle
comment: 


In [41]:
for block in blocks:
    print(block.keys())

dict_keys(['blockType', 'comment', 'creationDateTime', 'creator', 'id', 'isArchived', 'isEnabled', 'isEntireWell', 'isLocked', 'magnification', 'modificationDateTime', 'modifier', 'name', 'owner', 'reagents', 'scanType', 'version'])
dict_keys(['blockType', 'comment', 'creationDateTime', 'creator', 'id', 'isArchived', 'isEnabled', 'isLocked', 'modificationDateTime', 'modifier', 'name', 'owner', 'version'])
dict_keys(['blockType', 'comment', 'creationDateTime', 'creator', 'id', 'isArchived', 'isEnabled', 'isEntireWell', 'isLocked', 'magnification', 'modificationDateTime', 'modifier', 'name', 'owner', 'reagents', 'scanType', 'version'])
dict_keys(['blockType', 'comment', 'creationDateTime', 'creator', 'id', 'isArchived', 'isEnabled', 'isLocked', 'modificationDateTime', 'modifier', 'name', 'owner', 'photos', 'realease', 'version'])
dict_keys(['blockType', 'comment', 'creationDateTime', 'creator', 'dilutionFactor', 'exposureTime', 'id', 'incubationTime', 'isArchived', 'isEnabled', 'isLocked

In [ ]:
for block in blocks:
    if 'reagents' in block:
        print(f"Block Name: {block['name']}")
        print(f"Reagents: {block['reagents']}")

Block Name: Scan
Reagents: {'DetectionChannel_1': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0, 'sensitivityMode': 0}, 'exposureType': 'Coefficient', 'timeCoefficient': 100}, 'fluorochromeType': 'FluorochromeType_DAPI', 'isEnabled': True}, 'DetectionChannel_2': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0, 'sensitivityMode': 0}, 'exposureType': 'Coefficient', 'timeCoefficient': 230}, 'fluorochromeType': 'FluorochromeType_FITC', 'isEnabled': False}, 'DetectionChannel_3': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0, 'sensitivityMode': 0}, 'exposureType': 'Coefficient', 'timeCoefficient': 330}, 'fluorochromeType': 'FluorochromeType_PE', 'isEnabled': False}, 'DetectionChannel_4': {'exposureTimeAndCoefficient': {'exposureTime': {'linearityMode': 0, 'sensitivityMode': 0}, 'exposureType': 'Coefficient', 'timeCoefficient': 100}, 'fluorochromeType': 'FluorochromeType_APC', 'isEnabled': False}, 'DetectionChannel_5': {'exposure

In [ ]:
separate column for height width shapre
    
    height and widt h10 and 19 whre did that come frm and we dont need it

what does ROI stand for